In [1]:
from pynucastro.rates import Library, RateFilter
from pynucastro.nucdata import Nucleus
from pynucastro.networks import PythonNetwork

In [2]:
full_lib = Library("reaclib_default2_20220329")

In [3]:
# Some other options include: ge64, te108
endpoint = Nucleus('ni56')

In [4]:
# Could introduce pp-chain nuclei (d, t, he3, be7, li7, etc.)

core_nuclei = ["p", "d", "he3", "he4", "li7", "be7", "be8", "b8", "c12",
               "n13", "n14", "n15", "o14", "o15", "o16", "o17", "o18",
               "f17", "f18", "f19", "f20", "ne18", "ne19", "ne20", "ne21"]
core_nuclei = list(map(Nucleus, core_nuclei))

In [5]:
core_lib = full_lib.linking_nuclei(core_nuclei)

In [6]:
def is_beta_plus(rate):
    """ Filter for beta+ decays (and electron captures). """
    
    if len(rate.products) != len(rate.reactants):
        return False
    if len(rate.reactants) != 1:
        return False
    
    react, = rate.reactants
    prod, = rate.products
    return prod.Z < react.Z

In [7]:
# Restrict the library to these 7 reaction types

# Forward rates
p_gamma = RateFilter(reactants="p", max_products=1, exact=False)
alpha_gamma = RateFilter(reactants="he4", max_products=1, exact=False)
alpha_p = RateFilter(reactants="he4", products="p", exact=False)

# Reverse and weak rates
gamma_p = RateFilter(products="p", max_reactants=1, exact=False)
gamma_alpha = RateFilter(products="he4", max_reactants=1, exact=False)
p_alpha = RateFilter(reactants="p", products="he4", exact=False)
beta_plus = RateFilter(filter_function=is_beta_plus)

# Compute reduced library
red_lib = full_lib.filter((p_gamma, alpha_gamma, alpha_p,
        gamma_p, gamma_alpha, p_alpha, beta_plus))

In [8]:
from collections import deque
        
def product_limiter():
    """
    This helps trim the library a bit by excluding rates with
    products with more protons than Ni, heavier than Ni56,
    or with relatively high / low neutron percentages. 
    """
    
    # Proton number bounds
    Zlo, Zhi = 6, endpoint.Z
    # Nucleon number bounds
    Alo, Ahi = 12, endpoint.A
    # Bounds on A / Z ratio to drop peripheral nuclei
    Rlo, Rhi = 1.69, 2.2
    
    def limit_products(r):
        
        meet_conds = \
        (
            (Zlo <= p.Z <= Zhi and
             Alo <= p.A <= Ahi and
             Rlo <= p.A / p.Z <= Rhi and
             p.nucbind is not None)
            or (p.Z, p.A) == (1, 1)
            or (p.Z, p.A) == (2, 4)
            for p in r.products
        )
        return all(meet_conds)
    
    return limit_products

In [9]:
red_lib = red_lib.filter(RateFilter(filter_function=product_limiter()))
final_lib = Library(rates=core_lib.get_rates())

seeds = [nuc for nuc in core_nuclei if nuc.A >= 12]
encountered = set(seeds) | {Nucleus("p"), Nucleus("he4")}
seeds = deque(seeds)

while seeds:
    
    # Get the new rates with seed as a reactant
    seed = seeds.popleft()
    filt = RateFilter(reactants=seed, exact=False)
    new_lib = red_lib.filter(filt)
    if new_lib is None: continue
    final_lib += new_lib
    
    # Append all unseen nuclei to the queue
    prod_set = set(p for r in new_lib.get_rates() for p in r.products)
    prod = sorted(prod_set - encountered)
    seeds.extend(prod)
    encountered.update(prod)
    
encountered = sorted(encountered)

In [10]:
# remove duplicate rates
dupes = final_lib.find_duplicate_links()
for d in dupes:
    # if wc17 is present, then remove all other duplicate rates
    labels = {r.label: r for r in d}
    if len(labels) == len(d) and "wc17" in labels:
        for r in d:
            if r.label == "wc17":
                continue
            final_lib.remove_rate(r)
        r = labels["wc17"]
        print(f"Found rate {r} named {r.fname} with {len(d)} entries in the Library.")
        print(f"Kept only entry with label {r.label} out of {list(labels.keys())}.")


In [11]:
final_lib

p + p + e⁻ ⟶ H2 + 𝜈            [Q =   1.44 MeV] (p + p --> d <ec_reaclib_weak_>)
p + p ⟶ H2 + e⁺ + 𝜈            [Q =   1.44 MeV] (p + p --> d <bet+_reaclib_weak_>)
H2 + p ⟶ He3 + 𝛾               [Q =   5.49 MeV] (d + p --> He3 <de04_reaclib__>)
H2 + H2 ⟶ He4 + 𝛾              [Q =  23.85 MeV] (d + d --> He4 <nacr_reaclib__>)
He3 + p ⟶ He4 + e⁺ + 𝜈         [Q =  19.80 MeV] (He3 + p --> He4 <bet+_reaclib_weak_>)
He3 + H2 ⟶ p + He4             [Q =  18.35 MeV] (He3 + d --> p + He4 <de04_reaclib__>)
He3 + He3 ⟶ p + p + He4        [Q =  12.86 MeV] (He3 + He3 --> p + p + He4 <nacr_reaclib__>)
He3 + He4 ⟶ Be7 + 𝛾            [Q =   1.59 MeV] (He3 + He4 --> Be7 <cd08_reaclib__>)
3 He4 ⟶ C12 + 𝛾                [Q =   7.28 MeV] (3 He4 --> C12 <fy05_reaclib__>)
Li7 + p ⟶ He4 + He4            [Q =  17.35 MeV] (Li7 + p --> He4 + He4 <de04_reaclib__>)
Be7 + p ⟶ B8 + 𝛾               [Q =   0.14 MeV] (Be7 + p --> B8 <nacr_reaclib__>)
Be7 + H2 ⟶ p + He4 + He4       [Q =  16.77 MeV] (Be7 + d --> p + He4 +

In [12]:
encountered

[p,
 He4,
 C12,
 C13,
 N12,
 N13,
 N14,
 N15,
 O14,
 O15,
 O16,
 O17,
 O18,
 F16,
 F17,
 F18,
 F19,
 F20,
 Ne17,
 Ne18,
 Ne19,
 Ne20,
 Ne21,
 Ne22,
 Na19,
 Na20,
 Na21,
 Na22,
 Na23,
 Na24,
 Mg21,
 Mg22,
 Mg23,
 Mg24,
 Mg25,
 Mg26,
 Al22,
 Al23,
 Al24,
 Al25,
 Al26,
 Al27,
 Al28,
 Si24,
 Si25,
 Si26,
 Si27,
 Si28,
 Si29,
 Si30,
 P26,
 P27,
 P28,
 P29,
 P30,
 P31,
 P32,
 P33,
 S28,
 S29,
 S30,
 S31,
 S32,
 S33,
 S34,
 S35,
 Cl29,
 Cl30,
 Cl31,
 Cl32,
 Cl33,
 Cl34,
 Cl35,
 Cl36,
 Cl37,
 Ar31,
 Ar32,
 Ar33,
 Ar34,
 Ar35,
 Ar36,
 Ar37,
 Ar38,
 Ar39,
 K33,
 K34,
 K35,
 K36,
 K37,
 K38,
 K39,
 K40,
 K41,
 Ca34,
 Ca35,
 Ca36,
 Ca37,
 Ca38,
 Ca39,
 Ca40,
 Ca41,
 Ca42,
 Ca43,
 Ca44,
 Sc36,
 Sc37,
 Sc38,
 Sc39,
 Sc40,
 Sc41,
 Sc42,
 Sc43,
 Sc44,
 Sc45,
 Sc46,
 Ti38,
 Ti39,
 Ti40,
 Ti41,
 Ti42,
 Ti43,
 Ti44,
 Ti45,
 Ti46,
 Ti47,
 Ti48,
 V39,
 V40,
 V41,
 V42,
 V43,
 V44,
 V45,
 V46,
 V47,
 V48,
 V49,
 V50,
 Cr41,
 Cr42,
 Cr43,
 Cr44,
 Cr45,
 Cr46,
 Cr47,
 Cr48,
 Cr49,
 Cr50,
 Cr51,
 Cr52,
 Mn43,


In [13]:
rp_net = PythonNetwork(libraries=[final_lib])

In [14]:
# rp_net.write_network()